In [1]:
from langchain import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import PromptTemplate
from langchain.callbacks import WandbCallbackHandler, StdOutCallbackHandler
from langchain.callbacks.base import CallbackManager
from getpass import getpass

import os 
from os.path import join, dirname 


PATH_REPO = dirname(os.getcwd())
PATH_DATA = join(PATH_REPO, 'data')
PATH_UTILS = join(PATH_REPO, 'src')

import sys

sys.path.append(PATH_REPO)
#sys.path.append(PATH_UTILS)

from src import time_series
from src import utils_plot
from src import anomaly
from src import preprocess
from src import summarize
from src import top_terms_extractor
from src import top_trends
from src import preprocesser
import pandas as pd
import numpy as np
from datetime import datetime


In [2]:
import os

#os.environ["WANDB_API_KEY"] = 
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)


In [4]:
"""Main function.

This function is used to test the callback handler.
Scenarios:
1. OpenAI LLM
2. Chain with multiple SubChains on multiple generations
3. Agent with Tools
"""
session_group = datetime.now().strftime("%m.%d.%Y_%H.%M.%S")
wandb_callback = WandbCallbackHandler(
    job_type="inference",
    project="langchain_callback_demo",
    group=f"minimal_{session_group}",
    name="llm",
    tags=["test"],
)
manager = CallbackManager([StdOutCallbackHandler(), wandb_callback])
llm = OpenAI(temperature=0, callback_manager=manager, verbose=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bonells96. Use `wandb login --relogin` to force relogin


wandb: WARNING The wandb callback is currently in beta and is subject to change based on updates to `langchain`. Please report any issues to https://github.com/wandb/wandb/issues with the tag `langchain`.


In [5]:
name_data =  'Congo virus - Pakistan - Data (1)'#'Congo virus - Pakistan - Data (1)'
data = pd.read_csv(join(PATH_DATA, f'{name_data}.csv'))
#data['date'] = pd.to_datetime(data['date'])
data.head(5)

,id,text,translated_text,created_at,lang,source,name,screen_name,bio,us_created_at,...,friends_count,statuses_count,favourites_count,country,country_code,locality,region,sub_region,coordinates,date
0,1531879580767772672,Believe me this is not an online shoe store. I...,Believe me this is not an online shoe store. I...,2022-06-01T06:05:27+00:00,en,twitter,Shkaib Ahmad,@ShkmPro,MBBS 21' | Ghazian |Manchester United 🔴 | Foot...,2020-05-29T16:25:07+00:00,...,467,942,1964,Pakistan,PK,NaN,NaN,NaN,"30.0, 70.0",2022-06-01
1,1532683704706924546,Crimean-Congo Hemorrhagic Fever in Iraq https:...,Crimean-Congo Hemorrhagic Fever in Iraq https:...,2022-06-03T11:20:45+00:00,en,twitter,Dr Ali Raza Majid,@dralirazamajid,"Consultant Medical Virologist, Indus Hospital ...",2015-11-11T11:06:12+00:00,...,96,113,1107,Pakistan,PK,Peshāwar,North-West Frontier Province,NaN,"34.008, 71.57849",2022-06-03
2,1533333043934801921,the 3 phases of #denguefever. it is vital to l...,the 3 phases of #denguefever. it is vital to l...,2022-06-05T06:21:00+00:00,en,twitter,Ahmed Yanaal,@ahmed_yanaal,"🇲🇻. 👨‍⚕️. 🏃. @funnybeys, @lfc, and #medtwitter.",2019-12-04T14:40:50+00:00,...,677,3348,21942,Pakistan,PK,Karachi,Sindh,Karāchi District,"24.9056, 67.0822",2022-06-05
3,1534261731857354752,عید الاضحیٰ کے لیے قربانی کے جانور خریدتے وقت ...,Take special precautions when buying the sacri...,2022-06-07T19:51:16+00:00,ur,twitter,Hamid ur Rehman,@Hamidurrehmaan,"Health Reporter @SAMAATV , Past @DunyaNews, @...",2010-06-07T13:53:16+00:00,...,226,24258,32060,Pakistan,PK,Karachi,Sindh,Karāchi District,"24.9056, 67.0822",2022-06-07
4,1534255578817740802,ان دنوں میں کانگو وائرس اور لمپی اسکن کے کیسز ...,In these days the Congo virus and Lumpy Skin c...,2022-06-07T19:26:49+00:00,ur,twitter,SAMAA TV,@SAMAATV,پاکستان کا سب سے بڑانیوزچینل جولاتا ہے تازہ تر...,2009-02-27T10:38:25+00:00,...,77,350040,209,Pakistan,PK,NaN,NaN,NaN,"30.0, 70.0",2022-06-07


In [6]:
texts = data.loc[:,'text'].values.tolist()
texts_joined = ".\n".join(texts)

In [7]:
llm.get_num_tokens('.\n'.join(texts))


48971

In [10]:
llm_result = llm.generate(["Tell me a joke", "Tell me a poem"] * 3)
wandb_callback.flush_tracker(llm, name="simple_sequential")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: Network error (ConnectionError), entering retry loop.


In [11]:
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=2500, chunk_overlap=500)
docs = text_splitter.create_documents([texts_joined])

In [12]:
num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

In [13]:
num_tokens_first_doc

2368

In [20]:

summary_chain = load_summarize_chain(llm=llm, chain_type='map_reduce', callback_manager=manager
#                                      verbose=True # Set verbose=True if you want to see the prompts being used
                                    )

In [21]:
len(docs)

37

In [22]:
output = summary_chain.run(docs)
output
wandb_callback.flush_tracker(summary_chain, name="summarizer")

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


wandb: Network error (ConnectionError), entering retry loop.


In [17]:
output

' The National Institutes of Health has issued an advisory to take precautionary measures against the Congo virus during the Eid-ul-Azha festival. The government of Pakistan has taken precautionary measures to curb the spread of the virus, such as providing financial aid, open market and stock market support, and providing medical supplies and vaccines. The Khyber Pakhtunkhwa government has also issued an advisory to protect against the possible outbreak of Congo Virus in the Province. Symptoms of the virus are similar to those of dengue fever and preventive measures are being taken to contain the spread of the virus.'

In [33]:

map_prompt = """
Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])